In [25]:
from selenium.webdriver.chrome.options import Options
from os import path, listdir, rename, getcwd
from fake_useragent import UserAgent
from dateutil.parser import parse
from selenium import webdriver
from bs4 import BeautifulSoup
from math import ceil
import pandas as pd
import numpy as np
import requests
import pickle
import time

In [2]:
def chrome_user_agent():
    return UserAgent().chrome

chrome_user_agent()

'Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.3319.102 Safari/537.36'

In [3]:
def chrome_driver():
    options = Options()
    # options.add_argument('--headless')
    options.add_argument("window-size=1600,1080")
    options.add_argument(f'user-agent={chrome_user_agent()}')

    return webdriver.Chrome(options=options)

In [4]:
def socialblade(channel, driver):
    driver.get(f'https://socialblade.com/youtube/channel/{channel}')
    
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    time.sleep(np.random.randint(4,7))
    
    return soup.find('body').prettify()

In [5]:
def save_html(channel, soup):
    # open the file in w mode, set encoding to UTF-8
    with open(f"../download/socialblade/new/{channel}.html", "w") as f:
        f.write(str(soup))

In [6]:
def open_html(channel):
    with open(channel, 'r') as f:
        return f.read()

In [7]:
def check_channel(channel, channels_dict):
    # check if channel has been downloaded
    if channel not in channels_dict.keys():
        save_html(channel, socialblade(channel))
        channels_dict[channel] = 'downloaded'

    return channels_dict

In [8]:
def load_channels_dict():
    with open('../data/channels_dict', "rb") as f:
        channels_dict = pickle.load(f)
        return channels_dict

In [9]:
def save_channels_dict(channels_dict):
    # Save channels dictionary object
    with open('../data/channels_dict', "wb") as f:
        pickle.dump(channels_dict, f)

In [10]:
def reset_channels_dictionary():
    my_dict = load_channels_dict()

    for key in my_dict.keys():
        my_dict[key] = 'pending'

    for file in listdir('../download/socialblade/new'):
        my_dict[file[:-5]] = 'downloaded'

    for file in listdir('../download/socialblade/valid'):
        my_dict[file[:-5]] = 'valid'

    for file in listdir('../download/socialblade/invalid'):
        my_dict[file[:-5]] = 'invalid'

    save_channels_dict(my_dict)

In [11]:
my_dict = load_channels_dict()

p_counter, v_counter, i_counter, d_counter = 0,0,0,0

for key in my_dict.keys():
    if my_dict[key] == 'pending':
        p_counter += 1
    elif my_dict[key] == 'valid':
        v_counter += 1
    elif my_dict[key] == 'invalid':
        i_counter += 1
    elif my_dict[key] == 'downloaded':
        d_counter += 1

print(f'pending - {p_counter}')
print(f'valid - {v_counter}')
print(f'invalid - {i_counter}')
print(f'downloaded - {d_counter}')

pending - 0
valid - 616
invalid - 19
downloaded - 877


In [12]:
def download_missing_channels():
    driver = chrome_driver()
    driver.get('https://socialblade.com')
    time.sleep(7)

    channels_dict = load_channels_dict()

    for channel in channels_dict.keys():
        if channels_dict[channel] == 'pending':
            save_html(channel, socialblade(channel, driver))

            last = channel

            channels_dict[channel] = 'downloaded'
            save_channels_dict(channels_dict)

    driver.close()

In [13]:
# download_missing_channels()

In [14]:
def process_channels():
    channels_dict = load_channels_dict()

    my_list = []

    for channel in channels_dict.keys():
        if channels_dict[channel] == 'downloaded':
            file_name = f'../download/socialblade/{channel}.html'

            channel_soup = BeautifulSoup(open_html(file_name))

            if channel_soup.find('div', {'id': 'graph-youtube-monthly-vidviews-container'}):
                channels_dict[channel] = 'valid'
        else:
            channels_dict[channel] = 'invalid'
    
    for channel in channels_dict.keys():
        if channels_dict[channel] == 'valid':
            file_name = f'../download/socialblade/{channel}.html'

            channel_soup = BeautifulSoup(open_html(file_name))

            top_info = channel_soup.findAll('div', {'class': 'YouTubeUserTopInfo'})
            
            name = channel_soup.find('div', {'id': 'YouTubeUserTopInfoBlockTop'}).find('h1').text.strip()
            uploads = int(top_info[0].findAll('span')[1].text.replace(',', ''))
            subscribers = top_info[1].findAll('span')[1].text.strip()
            total_views = int(top_info[2].findAll('span')[1].text.replace(',', ''))
            created = parse(top_info[5].findAll('span')[1].text.strip()).date()
            daily_avg_sub = channel_soup.find('div', {'id': 'averagedailysubs'}).text.strip().replace(',','')
            daily_avg_views = channel_soup.find('div', {'id': 'averagedailyviews'}).text.strip().replace(',','')
            
            my_list.append([channel, name, uploads, subscribers, total_views, created, daily_avg_sub, daily_avg_views])

    save_channels_dict(channels_dict)

    return my_list
    

In [15]:
df_columns = ['channel', 'name', 'uploads', 'subscribers', 'total_views', 'created', 'daily_avg_sub', 'daily_avg_views']
channels_df = pd.DataFrame(process_channels(), columns=df_columns).set_index('channel')

In [16]:
channels_df

,name,uploads,subscribers,total_views,created,daily_avg_sub,daily_avg_views
channel,,,,,,,
UCJ9UwnsAJu-b2mc4KVAANBw,RustyFoxeGaming,77,81,2818,2020-03-20,+1,+13
UCsQIVOWrNBUMXUOg112GnXw,JolliAllGenGamer,583,2.22K,1136845,2019-11-29,+11,+9143
UCFMUPSaXKSl-trspfXD59TQ,Hubba Noob,556,691K,77996396,2019-09-04,+1.2K,+166237
UCM2JmrhrdFs9tuF7wq4xTsw,Atomic Dude,204,35.7K,29723263,2019-07-08,+277,+272130
UCYtia5cQdXSWH7GLMWxo9UQ,JShay Translations,223,40.6K,28143726,2020-09-22,+97,+111455
...,...,...,...,...,...,...,...
UCCHzhLksM03P4ge18Qe78Ag,iDraker,32,58,1886,2019-10-12,+1,+4
UCODy2NntK7rUm4kWi8ulOQA,Djurren,43,2.51K,6422,2019-05-26,+3,+12
UCO-5hTPWJaV8uo7L_VwSC7g,Hanky,41,64.5K,2795902,2019-08-26,+7,+434


In [17]:
channels_df.daily_avg_sub.replace('--', '0', inplace=True)
channels_df.daily_avg_sub.replace('\+', '', inplace=True, regex=True)

channels_df.daily_avg_views.replace('--', '0', inplace=True)
channels_df.daily_avg_views.replace('\+', '', inplace=True, regex=True)

In [18]:
channels_df['subscribers'] = channels_df['subscribers'].replace({'K': '*1e3', 'M': '*1e6'}, regex=True).map(pd.eval).astype(int)
channels_df['daily_avg_sub'] = channels_df['daily_avg_sub'].replace({'K': '*1e3'}, regex=True).map(pd.eval).astype(int)
channels_df['daily_avg_views'] = channels_df['daily_avg_views'].replace({'K': '*1e3'}, regex=True).map(pd.eval).astype(int)

In [19]:
channels_df['age'] = (pd.Timestamp.today().date() - channels_df.created).dt.days

In [20]:
channels_df = channels_df[['total_views', 'uploads', 'subscribers', 'daily_avg_sub', 'daily_avg_views', 'age']]

In [44]:
channels_df['views_per_video'] = (channels_df['total_views'] / channels_df['uploads']).round(1)

In [42]:
channels_df['weekly_videos'] = ((channels_df['uploads'] / channels_df['age']) * 7).round(1)

In [48]:
# drop channels that have no video upload
channels_df = channels_df[channels_df.uploads != 0]

In [49]:
channels_df

,total_views,uploads,subscribers,daily_avg_sub,daily_avg_views,age,views_per_video,weekly_videos
channel,,,,,,,,
UCJ9UwnsAJu-b2mc4KVAANBw,2818,77,81,1,13,417,36.6,1.3
UCsQIVOWrNBUMXUOg112GnXw,1136845,583,2220,11,9143,529,1950.0,7.7
UCFMUPSaXKSl-trspfXD59TQ,77996396,556,691000,1200,166237,615,140281.3,6.3
UCM2JmrhrdFs9tuF7wq4xTsw,29723263,204,35700,277,272130,673,145702.3,2.1
UCYtia5cQdXSWH7GLMWxo9UQ,28143726,223,40600,97,111455,231,126205.0,6.8
...,...,...,...,...,...,...,...,...
UCCHzhLksM03P4ge18Qe78Ag,1886,32,58,1,4,577,58.9,0.4
UCODy2NntK7rUm4kWi8ulOQA,6422,43,2510,3,12,716,149.3,0.4
UCO-5hTPWJaV8uo7L_VwSC7g,2795902,41,64500,7,434,624,68192.7,0.5


In [50]:
def save_channels_df(df):
    # Save channels dictionary object
    with open('../data/channels_df', "wb") as f:
        pickle.dump(channels_df, f)

In [51]:
save_channels_df(channels_df)